In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [2]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0
1,2020-01-22,no,Kosovo,0
2,2020-01-22,no,Kuwait,0
3,2020-01-22,no,Kyrgyzstan,0
4,2020-01-22,no,Laos,0


In [3]:
test_data=pd_JH_data[((pd_JH_data['country']=='Bhutan')|
                      (pd_JH_data['country']=='Denmark'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [4]:
test_data.head()

,date,state,country,confirmed
17012,2020-03-21,Faroe Islands,Denmark,92
17027,2020-03-21,Greenland,Denmark,2
17044,2020-03-21,no,Bhutan,2
17054,2020-03-21,no,Denmark,1326
17305,2020-03-22,Greenland,Denmark,4


In [5]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Bhutan,2022-06-22,no,59674
Denmark,2022-06-22,no,3157930


In [6]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state         country                      
Faroe Islands Denmark 2022-06-22      34658
Greenland     Denmark 2022-06-22      11971
no            Bhutan  2022-06-22      59674
              Denmark 2022-06-22    3157930

In [8]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [9]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state          country        
Faroe Islands  Denmark  17012             NaN
                        17311             NaN
                        17582        8.333333
                        17867       33.809524
                        18166       17.714286
                                     ...     
no             Denmark  250469    5276.718593
                        250754            inf
                        251041    1950.294761
                        251326    1173.819964
                        251608    1539.844352
Name: confirmed, Length: 3296, dtype: float64

In [10]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [11]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR', 'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [12]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,Kosovo,0
2,2,2020-01-22,no,Kuwait,0
3,3,2020-01-22,no,Kyrgyzstan,0
4,4,2020-01-22,no,Laos,0


In [13]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,Kosovo,0,NaN
2,2,2020-01-22,no,Kuwait,0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0,NaN
4,4,2020-01-22,no,Laos,0,NaN


In [14]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [15]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [16]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0,NaN,0.0
4,4,2020-01-22,no,Laos,0,NaN,0.0


In [17]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
251650,no,Zimbabwe,250371,2336.015902
251651,no,Zimbabwe,250584,2958.492451
251652,no,Zimbabwe,250891,3573.679289
251653,no,Zimbabwe,251133,3915.123400
251654,no,Zimbabwe,251654,3590.726761


In [18]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
251650,251650,2022-06-22,no,Andorra,43774,2.680451e+02,43644.0,670.446154
251651,251651,2022-06-22,no,Algeria,266006,2.533283e+04,266003.6,30226.681818
251652,251652,2022-06-22,no,Albania,277940,1.119687e+03,277889.8,1499.484881
251653,251653,2022-06-22,no,Argentina,9341492,inf,9347099.8,1665.803345
251654,251654,2022-06-22,no,Zimbabwe,255049,2.338835e+03,255012.6,3590.726761


In [19]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [20]:
pd_result_larg[pd_result_larg['country']=='Denmark'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
251296,251296,2022-06-21,Faroe Islands,Denmark,34658,inf,34658.0,9.526718e+15
251326,251326,2022-06-21,no,Denmark,3155972,1173.819964,3155789.7,1.818005e+03
251574,251574,2022-06-22,Greenland,Denmark,11971,inf,11971.0,2.794369e+31
251580,251580,2022-06-22,Faroe Islands,Denmark,34658,inf,34658.0,-2.022539e+31
251608,251608,2022-06-22,no,Denmark,3157930,1539.844352,3157793.2,1.575138e+03


In [21]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)